__Imports__

In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from google_drive_downloader import GoogleDriveDownloader as gdd

__Download File from link given in Canvas__

This will be stored into your local...Do not add into git, file is too large to be pushed onto git master branch

So, we download locally

In [2]:
gdd.download_file_from_google_drive(file_id='0B04GJPshIjmPRnZManQwWEdTZjg',
                                    dest_path='/Users/mwoo/Downloads/trainingandtestdata.zip',
                                    unzip=True)

In [3]:
# gdd.download_file_from_google_drive(file_id='0B04GJPshIjmPRnZManQwWEdTZjg',
#                                     dest_path='/Users/swapnilbasu/Downloads/trainingandtestdata.zip',
#                                     unzip=True)

__Create spark session object (Data Processing)__

In [4]:
spark=SparkSession.builder.appName('data_processing').getOrCreate()

__Load in data__

In [5]:
training_data = spark.read.csv("/Users/mwoo/Downloads/training.1600000.processed.noemoticon.csv",header=False)

__Renaming columns__

In [6]:
training_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5']

In [7]:
training_data = training_data.toDF("target",'id','date','query','user_name','text')

In [8]:
training_data.columns

['target', 'id', 'date', 'query', 'user_name', 'text']

__Selecting the target value and text__

In [9]:
df = training_data.select('text','target')

In [10]:
df.show(5)

+--------------------+------+
|                text|target|
+--------------------+------+
|@switchfoot http:...|     0|
|is upset that he ...|     0|
|@Kenichan I dived...|     0|
|my whole body fee...|     0|
|@nationwideclass ...|     0|
+--------------------+------+
only showing top 5 rows



In [11]:
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- target: string (nullable = true)



We can see below that its an even split between positive and negative tweets

0: negative
4: positive

In [12]:
from pyspark.sql.functions import col
df.groupBy("target").count().orderBy(col("count").desc()).show()

+------+------+
|target| count|
+------+------+
|     0|800000|
|     4|800000|
+------+------+



__Model Pipeline__

In [13]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

__Regular Expression Tokenizer__

In [14]:
regexTokenizer = RegexTokenizer(inputCol="text", 
                                outputCol="words", 
                                pattern="\\W")

__Stop Words Download from NLTK__

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mwoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

__Stop Words Remover__

In [16]:
from nltk.corpus import stopwords
import string
sp = set(string.punctuation)
stop_words = set(stopwords.words('english'))
extra_words = {"http","https","amp","rt","t","c","the"}
for i in extra_words:
    stop_words.add(i) 
stop_words = list(stop_words)
stopwordsRemover = StopWordsRemover(inputCol="words", 
                                    outputCol="filtered").setStopWords(stop_words)

__Bag of words count__

This is a type of feature engineering

In [17]:
countVectors = CountVectorizer(inputCol="filtered", 
                               outputCol="features", 
                               vocabSize=10000, minDF=5)

__StringIndexer__

This is where we create our new dataframe in spark

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents
pipelineFit = pipeline.fit(df)
dataset = pipelineFit.transform(df)
dataset.show(5)

+--------------------+------+--------------------+--------------------+--------------------+-----+
|                text|target|               words|            filtered|            features|label|
+--------------------+------+--------------------+--------------------+--------------------+-----+
|@switchfoot http:...|     0|[switchfoot, http...|[switchfoot, twit...|(10000,[1,10,16,6...|  0.0|
|is upset that he ...|     0|[is, upset, that,...|[upset, update, f...|(10000,[6,70,172,...|  0.0|
|@Kenichan I dived...|     0|[kenichan, i, div...|[kenichan, dived,...|(10000,[4,213,251...|  0.0|
|my whole body fee...|     0|[my, whole, body,...|[whole, body, fee...|(10000,[3,325,374...|  0.0|
|@nationwideclass ...|     0|[nationwideclass,...|[nationwideclass,...|(10000,[20,486],[...|  0.0|
+--------------------+------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



__Set seed for reproducibility__

In [19]:
# (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

In [20]:
trainingData = dataset
print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1600000


In [21]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
# predictions = lrModel.transform(testData)
# predictions.filter(predictions['prediction'] == 0) \
#     .select("text","probability","label","prediction") \
#     .orderBy("probability", ascending=False) \
#     .show(n = 10, truncate = 30)

In [22]:
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
# evaluator.evaluate(predictions)

__Format Test Data__

In [23]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream

ACCESS_TOKEN = "1458842253779161088-QFeO6udaAdHR4VARxaDza1w4LUlooE"
ACCESS_TOKEN_SECRET = "tC7IJDbl5T97Zvu3kE8sdGnmZWC2qxOrkdOv90YkdzIVO"
API_KEY = "KLP5ct26qaVo0KjAgP8O4j4y5"
API_KEY_SECRET = "AbxH3913WIPG0FHIwvVRomul92RWvuOdxRo2ecXR6H0Qgibo29"

auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

# public_tweets = api.home_timeline()
# for tweet in public_tweets:
#     print(tweet.text)
    
# user = api.get_user(screen_name='twitter')
# print(user.screen_name)
# print(user.followers_count)
# for friend in user.friends():
#     print(friend.screen_name)


In [24]:
tweet_list = list()
# Subclass Stream to print IDs of Tweets received
class IDPrinter(tweepy.Stream):
    
    def on_status(self, status):
        tweet_list.append(status.text)
        #print(tweet_list)
        print(status.text)
        if len(tweet_list) == 10:
            Stream.disconnect(self)
# Initialize instance of the subclass
printer = IDPrinter(
  API_KEY, API_KEY_SECRET,
  ACCESS_TOKEN, ACCESS_TOKEN_SECRET
)

# Filter realtime Tweets by keyword
# printer.filter(track=["Spiderman"])
printer.sample(languages=['en'])

ㅤ

ROCK— COLLECTION?!

         Sorry man, but a guy like me 
       can’t be seen around... yeah... 
      you get… https://t.co/kmmbiJA6KP
Check out the newest cardigan in the shop! https://t.co/B1Hchox08X


Stream connection closed by Twitter


for the record i dont feel comfortable having a private
@cousindolores ᅠ
We can eat your dessert after the dinner.
RT @Julie_Hugginz: @CBSLA I'd rather be a fully-vaccinated sick person than un-vaccinated dead one.
RT @guccidobi: introducing kim doyoung and so junghwan, the antagonist of "the mysterious class" https://t.co/x0rLpqNV4F
RT @EvaMooreXO: Sending a sexy video 💦 to all my December subscribers on OF today!  See you there 😉
I’m available for incall/outcall from M…
RT @LostInThePondUS: Oh no! Tonight's video is refusing to render. To borrow an Americanism, I think we're going to have to take a rainchec…
@ffsidkwhy FOR REAL 😭
RT @WrestleOps: Whenever Naomi is on my screen she always has my full attention and I am always excited to see literally anything she does.…


In [32]:
df_2 = pd.DataFrame(np.array(tweet_list))

In [33]:
df_2.columns = ['text']

In [34]:
df_2.head()

,text
0,"ㅤ\n\nROCK— COLLECTION?!\n\n Sorry man,..."
1,Check out the newest cardigan in the shop! htt...
2,for the record i dont feel comfortable having ...
3,@cousindolores ᅠ\nWe can eat your dessert afte...
4,RT @Julie_Hugginz: @CBSLA I'd rather be a full...


In [35]:
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext
df_2 = spark.createDataFrame(df_2)

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol= "text", outputCol='label')
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors,label_stringIdx])
# Fit the pipeline to training documents
pipelineFit = pipeline.fit(df_2)
dataset = pipelineFit.transform(df_2)
dataset.show(5)

+---------------------+--------------------+--------------------+---------+-----+
|                 text|               words|            filtered| features|label|
+---------------------+--------------------+--------------------+---------+-----+
|ㅤ\n\nROCK— COLLEC...|[rock, collection...|[rock, collection...|(0,[],[])|  9.0|
| Check out the new...|[check, out, the,...|[check, newest, c...|(0,[],[])|  2.0|
| for the record i ...|[for, the, record...|[record, dont, fe...|(0,[],[])|  8.0|
| @cousindolores ᅠ\...|[cousindolores, w...|[cousindolores, e...|(0,[],[])|  0.0|
| RT @Julie_Hugginz...|[rt, julie_huggin...|[julie_hugginz, c...|(0,[],[])|  4.0|
+---------------------+--------------------+--------------------+---------+-----+
only showing top 5 rows



In [37]:
testData = dataset

In [43]:
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] = 0) \
    .select("text","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

Py4JJavaError: An error occurred while calling o1032.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 61.0 failed 1 times, most recent failure: Lost task 4.0 in stage 61.0 (TID 275) (michaels-air-3 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (ProbabilisticClassificationModel$$Lambda$4567/0x0000000802219040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.j

In [42]:
predictions.show()

Py4JJavaError: An error occurred while calling o940.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 60.0 failed 1 times, most recent failure: Lost task 0.0 in stage 60.0 (TID 270) (michaels-air-3 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (ProbabilisticClassificationModel$$Lambda$4567/0x0000000802219040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterat